In [1]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

## 1) Prepare URL

In [93]:
def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url

### 1a) Get Clinical Informatics Jobs

In [94]:
url = get_url('Clinical Informatics', 'USA')
print(url)

https://www.indeed.com/jobs?q=Clinical+Informatics&l=USA


### 1b) Get Health Informatics Jobs

In [95]:
url = get_url('Health Informatics', 'USA')
print(url)

https://www.indeed.com/jobs?q=Health+Informatics&l=USA


## 2) Search Jobs

In [99]:
response = requests.get(url)
#print(response)
soup = BeautifulSoup(response.text, 'html.parser')
#print(soup)#job_seen_beacon
cards = soup.find_all('div', 'job_seen_beacon')
#print(cards)
print(len(cards))

15


## 3) Get job post

In [7]:
def get_record(card):
    """Extract job data from a single record"""
    job_header= card.table.tbody.tr.td
    job_title = job_header.div.h2.a.span.get('title')
    job_link= 'https://www.indeed.com'+job_header.div.h2.a.get('href')

    try:
      job_company=job_header.find('span','companyName').text.strip()
    except:
      print("No company name found for job "+job_title)

    try:
      job_ratings=job_header.find('span','ratingsDisplay').text.strip()
    except:
      print("No job ratings found for job "+job_title)
      job_ratings=""

    try:
      job_location=job_header.find('div','companyLocation').text.strip()
    except:
      print("No job location found for job "+job_title)

    try:
      job_date=card.find('span','date').text.strip().replace("Posted", "").replace("EmployerActive", "").strip()
      print(job_date)
    except:
      print("No job date found for job "+job_title)

    try:
      job_snippets=card.find_all('div','attribute_snippet')
      print(len(job_snippets))
    except:
      #print("No job location found for job "+job_title)
      job_snippets=""

    if job_snippets!="":
        try:      
          job_salary=job_snippets.find('svg',{'aria-label':'Salary'})
        except:
          #print("No job salary found for job "+job_title)
          job_salary=""
        try:      
          job_type=job_snippets.find('svg',{'aria-label':'Job type'})
        except:
          #print("No job type found for job "+job_title)
          job_type=""
        try:      
          job_shift=job_snippets.find('svg',{'aria-label':'Shift'})
        except:
          #print("No job shift found for job "+job_title)
          job_shift=""
    else:
        job_salary=""
        job_type=""
        job_shift=""

    try:      
        job_brief=card.find('div','job-snippet').ul.li.text.strip().replace('\n', ' ')
    except:
        job_brief=""
    
    print(job_brief)

    collect_date = datetime.today().strftime('%Y-%m-%d')     
    record = (job_title, job_link, job_company, job_location, job_date, collect_date, job_brief, job_ratings,job_salary, job_type, job_shift)
    return record

## 4) Get all jobs by scraping the next pages of the search

In [ ]:
records=[]
url=get_url('Health Informatics', 'USA') 
#url=get_url('Clinical Informatics', 'USA') # Uncomment and comment the above line to retrieve the Health Informatics Jobs
for i in range(67):
    print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div', 'job_seen_beacon')

    for card in cards:
        record = get_record(card)
        if record in records:
            continue
        else:
            records.append(record)
    try: 
        url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
    except AttributeError:
        print('search results error in iter '+str(i))
        break

In [26]:
print(len(records))

831


## 5) Save job posts in file

In [27]:
# save the job data
with open('results.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['JobTitle', 'JobUrl','Company', 'Location', 'PostDate', 'ExtractDate', 'Summary','Job_ratings', 'Salary', 'JobType','JobShift'])
    writer.writerows(records)

## 6) Getting jobs briefing: title, URL, summary, location, etc from the search result of the previous steps

In [6]:
from selenium import webdriver
import pandas as pd
jobs = pd.read_csv('results.csv')
titles = jobs['JobTitle'].tolist()
urls = jobs['JobUrl'].tolist()
company = jobs['Company'].tolist()
location= jobs['Company'].tolist()
PostDate= jobs['PostDate'].tolist()
ExtractDate= jobs['ExtractDate'].tolist()
Summary= jobs['Summary'].tolist()
Job_ratings= jobs['Job_ratings'].tolist()
Salary= jobs['Salary'].tolist()
JobType= jobs['JobType'].tolist()
JobShift= jobs['JobShift'].tolist()

In [17]:
# Installing the web driver for web browser
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [18]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST chromedriver version for 101.0.4951 google-chrome
[WDM] - There is no [mac64] chromedriver for browser 101.0.4951 in cache
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/aelnosho/.wdm/drivers/chromedriver/mac64/101.0.4951.41]
/var/folders/7v/g_t4898131v7f09flvsqg0v80000gn/T/ipykernel_1115/2160715986.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [100]:
import os
# The below is the path of the web browser driver on the machine used for this research.
# To reuse, change the below file location to the location of the web browser driver on your local machine
os.path.exists('/Users/aelnosho/.wdm/drivers/chromedriver/mac64/101.0.4951.41')

True

## 7) Getting jobs details by opening each job URL

In [ ]:
import sys
import time
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
count=0
jobsdesc=[]
for l in urls:
    url = l
    # initiating the webdriver. Parameter includes the path of the webdriver.
    driver = webdriver.Chrome('/Users/aelnosho/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver') 
    try:
        driver.get(url) 

        # this is just to ensure that the page is loaded
        time.sleep(5) 

        html = driver.page_source
        # this renders the JS code and stores all
        # of the information in static HTML code.

        # Now, we could simply apply bs4 to html variable
        soup = BeautifulSoup(html, "html.parser")

        try:
            salary = soup.find('div', {'id' : 'salaryGuide'}).ul.text
            Salary[count]=salary
        except:
            print("No salary found for job ")
        jobDescription = soup.find('div', {'id' : 'jobDescriptionText'}).text
        #print(jobDescription)
        jobsdesc.append(jobDescription)
    except:
        print("Description of job number "+str(count)+" was not retrieved")
        jobsdesc.append("")
    count=count+1
    
driver.close() # closing the webdriver

In [33]:
print(len(jobsdesc))

571


In [8]:
## Checking for error loading the job descriptions pages detail
diff=len(urls)-count
print(diff)
print(count)

0
831


## 8) load remaining jobs that did not load in previous point due to error loading

In [ ]:
for i in range(count, len(jobsdesc)):
    url=urls[u]
    print(url)
    driver = webdriver.Chrome('/Users/aelnosho/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver') 
    driver.get(url) 

    # this is just to ensure that the page is loaded
    time.sleep(5) 

    html = driver.page_source
    # this renders the JS code and stores all
    # of the information in static HTML code.

    # Now, we could simply apply bs4 to html variable
    soup = BeautifulSoup(html, "html.parser")
    
    try:
        salary = soup.find('div', {'id' : 'salaryGuide'}).ul.text
        Salary[count]=salary
    except:
        print("No salary found for job ")
        
    jobDescription = soup.find('div', {'id' : 'jobDescriptionText'}).text
    jobsdesc.append(jobDescription)
    count=count+1

## 9) save jobs with job description into csv

In [29]:
df_updated = pd.DataFrame({'Job_titles':titles,'Job_URLS':urls,'Company':company,'location':location,'PostDate':PostDate,'ExtractDate':ExtractDate,'Summary':Summary,'Job_ratings':Job_ratings,'Job_description':jobsdesc,'Salary':Salary,'JobType':JobType,'JobShift':JobShift})
df_updated.to_csv('JobsDetails.csv')

## 10) Analyze jobs

### 10A) Clean the job description

In [16]:
import pandas as pd
df_updated = pd.read_csv(r'JobDetailsAnalysis-HI.csv')

In [17]:
from html.parser import HTMLParser
import re
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def cleanURLEmail(strip):
    # extract emails
    match = re.findall(r'[\w\.-]+@[\w\.-]+', strip)
    temp = strip
    for m in match:
        temp = strip.replace(m, '', 100)
        strip = temp
    match3 = re.findall(r'^https?:\/\/.*[\r\n]*', strip)
    for m in match3:
        temp = strip.replace(m, '', 100)
        strip = temp
    return strip


#Remove Stop words, punctuation, and extract lemmatization, and stemming.
def clean(doc,stop,exclude):
    lemma = WordNetLemmatizer()
    stop_free = " ".join([i for i in str(doc).lower().split() if i not in stop])
    no_punc = map(lambda x: re.sub("[^a-z]", " ", x), stop_free)
    punc_free = "".join(ch for ch in no_punc if ch not in exclude)
    normalized=""
    stemmed = ""
    for word in punc_free.split():
        try:
            lWord=lemma.lemmatize(word)
        except:
            continue
        normalized=normalized+lWord+" "
    stop_free2 = " ".join([i for i in normalized.lower().split() if i not in stop])
    return stop_free2

In [18]:
from selenium import webdriver
import time
import csv
import pandas as pd
#import simplejson
from nltk import ngrams
import datetime
import re
import string
import nltk
import requests
from nltk.corpus import stopwords
import unicodedata



driver = webdriver.Chrome('/Users/aelnosho/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver') 
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}
statesBSC={}
statesMaster={}
statesCertifications={}
states={}
for k, v in us_state_abbrev.items():
    statesBSC[v]=0
    statesMaster[v] = 0
    statesCertifications[v] = 0
    states[v]=0


def cleanJobDescription(description):
    doc_complete=[]
    doc_clean=[]
    tList=[]
    exclude = set(string.punctuation)
    stop = set(stopwords.words('english'))
    lines = open("stop3").read().splitlines()
    for word in lines:
        #print(word)
        stop.add(word)
    myStop = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all",
                 "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst",
                 "amoungst", "amount", "an", "and", "another", "any", "anyhow", "anyone", "anything", "anyway",
                 "anywhere", "are", "around", "as", "at", "back", "be", "became", "because", "become", "becomes",
                 "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides",
                 "between", "beyond", "bill", "both", "bottom", "but", "by", "call", "can", "cannot", "cant", "co",
                 "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due",
                 "during", "each", "eg", "eight", "either", "eleven", "else", "elsewhere", "empty", "enough", "etc",
                 "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen",
                 "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found",
                 "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have",
                 "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself",
                 "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed",
                 "interest", "into", "itself", "keep", "last", "latter", "latterly", "least",
                 "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more",
                 "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither",
                 "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing",
                 "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other",
                 "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps",
                 "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious",
                 "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some",
                 "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system",
                 "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there",
                 "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin",
                 "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to",
                 "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until",
                 "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when",
                 "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon",
                 "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose",
                 "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself",
                 "yourselves", "the", 'http', 'https', '://', 'www', 'com', '8800', '...', '....','//','/','..', 'yep', '.).', '](#', '.:).',
                '++..', 'github', 'etc', 'also', 'org', 'gee', 'let', 'know', 'ever',
                'vcntr', 'falseamount', 'isig']
    for word in myStop:
        stop.add(word)
        
    #strip HTML tags from tweet
    temp=""
    #strip = strip_tags(description)
    #strip1 = cleanURLEmail(description)
    strip1 = str(description).replace(".","")
    strip2 = clean(strip1,stop,'')
    return(strip2)



/var/folders/7v/g_t4898131v7f09flvsqg0v80000gn/T/ipykernel_932/3880222035.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/aelnosho/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver')


In [19]:
df_updated['cleanDesc']= df_updated.apply(lambda x : cleanJobDescription(x['Job_description']),axis=1)

### 10b) Get skills, certificates, training, and degrees

In [20]:
# Funcation to flag the job if it required or prefer a professional certification or a university degree
def checkDegrees(description,degreeCheck):
    result=0
    graduateKeywords=['mba','msc', 'md', 'master', 'phd', 'doctorate']
    bachelorKeywords = ['college', 'bachelor', 'bsc','ba','bs','aa']
    degrees=['mba','msc', 'md', 'master', 'phd', 'doctorate','college', 'bachelor', 'bsc','ba','bs','aa','degree']
    if degreeCheck == 'bachelor':
        l= bachelorKeywords   
    elif degreeCheck == 'graduate':
        l=graduateKeywords
    elif degreeCheck == 'college':
        l=degrees
    job=str(description).lower().split()
    for w in job:
        if w in l:
            result=1
            break
    return result

In [21]:
# Function that extracts the certifications, skills, programming languages, data visualization tools required 
# or prefered in the job description
# check to exclude certification if has any of the following degrees=['mba','msc', 'md', 'master', 'phd', 'doctorate','college', 'bachelor', 'bsc']
def getText(description, search):
    count = 0
    degree=0
    # For professional certifications exclude, any academic certification 
    degreesOrLiscence=['mba','msc', 'md', 'master', 'phd', 'doctorate','college', 'bachelor', 'bsc','ba','bs','aa','degree','license','licensure']
    try:
        TexttotSet=set()
        words=description.lower().split()
        count=0
        cert=0
        for w in words:
            temp=""
            if (search=="certifications" and ("ertified" in w or "ertificat" in w)) or (search=="skills" and "skill" in w) or (search=="trainings" and "train" in w)or (search=="degrees" and "degree" in w) or (search=="license" and ("license" in w or "licensure" in w)) or (search=="visualization" and "visualization" in w) or (search=="programming" and "progamming" in w):
                degree=0
                counter=4
                i=count-1
                tempQueue=[]
                while(i>-1 and counter>0):
                    if words[i] in degreesOrLiscence:
                        degree=1
                        break
                    tempQueue.append(words[i])
                    counter-=1
                    i-=1
                if degree==0:
                    while(len(tempQueue)>0):
                        temp=tempQueue.pop(0)+" "+temp
                counter=1
                while (degree==0 and counter<5 and counter+count<len(words)):
                    if words[count+counter] in degreesOrLiscence:
                        degree=1
                        break
                    temp=temp+" "+words[count+counter]
                    counter+=1
                if degree==0:
                    TexttotSet.add(temp)
            count+=1
        
        # Extract the certification or skill name that could consists of one word, two words, three words...upto six words
        s=""
        for t in TexttotSet:
            Text=str(t)
            #Bigrams
            bigramsText=""
            n = 2
            n_grams = ngrams(Text.split(), n)
            for grams in n_grams:
                temp=""
                temp=grams[0]+"_"+grams[1]
                bigramsText=bigramsText+" "+temp

            #trigrams
            trigramsText = ""
            n = 3
            n_grams = ngrams(Text.split(), n)
            for grams in n_grams:
                temp=grams[0]+"_"+grams[1]+"_"+grams[2]
                trigramsText = trigramsText + " " + temp

            fourgramsText = ""
            n = 4
            n_grams = ngrams(Text.split(), n)
            for grams in n_grams:
                temp=""
                temp=grams[0]+"_"+grams[1]+"_"+grams[2]+"_"+grams[3]
                fourgramsText = fourgramsText + " " + temp

            fivegramsText=""
            n = 5
            n_grams = ngrams(Text.split(), n)
            for grams in n_grams:
                temp=""
                temp=grams[0]+"_"+grams[1]+"_"+grams[2]+"_"+grams[3]+"_"+grams[4]
                fivegramsText = fivegramsText + " " + temp

            sixgramsText=""
            n = 6
            n_grams = ngrams(Text.split(), n)
            for grams in n_grams:
                temp=""
                temp=grams[0]+"_"+grams[1]+"_"+grams[2]+"_"+grams[3]+"_"+grams[4]+"_"+grams[5]
                sixgramsText = sixgramsText + " " + temp

            s=s+Text+" "+bigramsText+" "+trigramsText+" "+fourgramsText+" "+fivegramsText+" "+sixgramsText+" "
            return str(s.strip())
    except Exception as e:
        print(e)
        

In [22]:
df_updated['certifications']= df_updated.apply(lambda x : getText(x['cleanDesc'],'certifications'),axis=1)
df_updated['skills']= df_updated.apply(lambda x : getText(x['cleanDesc'],'skills'),axis=1)
df_updated['trainings']= df_updated.apply(lambda x : getText(x['cleanDesc'],'trainings'),axis=1)
df_updated['degrees']= df_updated.apply(lambda x : getText(x['cleanDesc'],'degrees'),axis=1)
df_updated['license']= df_updated.apply(lambda x : getText(x['cleanDesc'],'license'),axis=1)
df_updated['programming']= df_updated.apply(lambda x : getText(x['cleanDesc'],'programming'),axis=1)
df_updated['visualization']= df_updated.apply(lambda x : getText(x['cleanDesc'],'visualization'),axis=1)

In [23]:
df_updated['bachelor']=df_updated.apply(lambda x : checkDegrees(x['cleanDesc'],'bachelor'),axis=1)
df_updated['graduate']=df_updated.apply(lambda x : checkDegrees(x['cleanDesc'],'graduate'),axis=1)
df_updated['college']=df_updated.apply(lambda x : checkDegrees(x['cleanDesc'],'college'),axis=1)

In [29]:
#def checkCollege(bachelor,graduate):
#    if bachelor ==1 or graduate==1:
#        return 1
#    else:
#        return 0
#df_updated['college']=df_updated.apply(lambda x : checkCollege(x['bachelor'],x['graduate']),axis=1)

In [24]:
# Jobs with undergraduate or graduate degrees requested
df_updated[df_updated['college']==1]

,Unnamed: 0,Unnamed: 0.1,Job_titles,Job_URLS,Company,location,PostDate,ExtractDate,Summary,Job_ratings,...,otherDegree,trainings,degrees,remote,college,states,certificationFlag,license,programming,visualization
0,0,0,Clinical Informatics Specialist - Contractor Role,https://www.indeed.com/company/Galen-Healthcar...,Galen HealthCare Solutions,Remote,24 days ago,2022-05-23,Serves as a resource person for daily operatio...,2.8,...,1,program support end user assist application i...,None,1,1,0,0,None,None,None
1,1,1,Electronic Health Record (EHR) Adoption/Optimi...,https://www.indeed.com/company/DEEPMILE/jobs/E...,DEEPMILE,Remote,3 days ago,2022-05-23,Knowledge of various specialized areas of heal...,NaN,...,1,electronic health record ehr learning support...,None,1,1,0,0,None,None,None
2,2,2,Clinical Informatics,https://www.indeed.com/rc/clk?jk=e1a9ee41cd34a...,Innovacer,Remote,12 days ago,2022-05-23,We are looking for a Clinical Informatics who ...,NaN,...,1,None,None,1,1,0,1,None,None,None
3,3,3,Informatics & Training Analyst,https://www.indeed.com/rc/clk?jk=c37cd6c881a7a...,Centura Health,"Centennial, CO 80112+1 location",30+ days ago,2022-05-23,"Informatics and Training Analyst supports, com...",3.8,...,1,skill commitment greater informatics analyst ...,None,0,1,CO,0,None,None,None
4,4,4,Clinical Informatics Consultant (Generalist),https://www.indeed.com/company/Edera-L3C/jobs/...,Edera L3C,Remote,5 days ago,2022-05-23,The Clinical Informatics Consultant works dire...,NaN,...,1,implementation workflow development testing c...,capability education experience year healthca...,1,1,0,1,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,820,820,Epidemiology Program Manager (Masters),https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,PMHCC Inc.,"Philadelphia, PA+2 locations",30+ days ago,2022-05-23,"Epidemiology, statistics, or informatics: 7 ye...",3.8,...,0,covid response philadelphia coordinate dcc st...,None,0,1,PA,0,None,None,stream dcc produce report supply covid case d...
823,823,823,Senior Fullstack Software Developer,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Boston Children's Hospital,"Boston, MA+1 location",30+ days ago,2022-05-23,Building custom API servers to provide data to...,4.1,...,0,None,None,0,1,MA,0,None,None,None
824,824,824,"Data Engineer, Insurance , Full time permanent",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"Villanti and Company,","San Diego, CA",6 days ago,2022-05-23,Well established and growing Insurance Company...,NaN,...,0,None,quicksight tableau powerbi education computer...,0,1,CA,0,None,None,warehouse design experience data tool quicksi...
827,827,827,Senior Data Engineer - TS/SCI w/Poly,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,General Dynamics,"Remote in McLean, VA",30+ days ago,2022-05-23,"Bachelor’s degree in information systems, info...",3.8,...,1,None,None,1,1,VA,1,None,None,None


In [45]:
# Jobs with College graduate degrees requested
dfupdated1=df_updated[df_updated['cleanDesc'].str.contains(r'critical thinking', na=False)]
dfupdated1[df_updated['graduate']==1]
# Analytical problem solving
# 182+73 graduate
# 136+256 undergraduate

# Communication skill
# 130 graduate
# 248 undergraduate

# Oral skill
# 120 graduate
# 165 undergraduate

# project management skill
# 85 graduate
# 155 undergraduate

# statistics skill
# 117 graduate
# 141 undergraduate

# interpersonal skill
# 71 graduate
# 128 undergraduate

# critical thinking skill
# 26 graduate
# 49 undergraduate

/var/folders/7v/g_t4898131v7f09flvsqg0v80000gn/T/ipykernel_932/243140210.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfupdated1[df_updated['graduate']==1]


,Unnamed: 0,Unnamed: 0.1,Job_titles,Job_URLS,Company,location,PostDate,ExtractDate,Summary,Job_ratings,...,otherDegree,trainings,degrees,remote,college,states,certificationFlag,license,programming,visualization
16,16,16,Clinical Informatics Specialist,https://www.indeed.com/rc/clk?jk=49cf0b5880459...,Oregon Health & Science University,"Remote in Portland, OR 97239",15 days ago,2022-05-23,The department will apply the science of infor...,3.9,...,1,future state coordinate develops plan user do...,clinical healthcare environment associate rel...,1,1,OR,1,implement solution registration certification ...,None,None
39,39,39,Clinical Informatics Specialist,https://www.indeed.com/rc/clk?jk=455c2d5b2baf1...,University of California San Francisco,Remote,17 days ago,2022-05-23,Broadly encompassing or in-depth knowledge of ...,4.1,...,1,biostatistics equivalent combination experienc...,None,1,1,0,0,None,None,None
79,79,79,CHESS: Clinical Informatics Analyst,https://www.indeed.com/rc/clk?jk=ec9aca1a1cb77...,MedCost LLC,Remote,4 days ago,2022-05-23,Familiar with clinical and administrative work...,3.1,...,1,standardized ontology experience testing impl...,None,1,1,0,0,None,None,database data sourcing blending technique exp...
160,160,160,Sr. Business Intelligence Analyst - Pop. Healt...,https://www.indeed.com/rc/clk?jk=37a8fbd067108...,Johns Hopkins Health System,"Baltimore, MD 21201 (Downtown area)+1 location",30+ days ago,2022-05-23,Advanced knowledge of medical terminology and ...,3.4,...,0,None,None,0,1,MD,1,strengthen team success required certificatio...,None,data collection reporting data information di...
199,199,199,Online Part-Time Faculty - Health Services Adm...,https://www.indeed.com/rc/clk?jk=1ceaa1fd44d58...,Bryant & Stratton College,"Orchard Park, NY 14127",30+ days ago,2022-05-23,Bryant & Stratton College Online Division is s...,3.7,...,0,None,ability support student stage skill knowledge...,0,1,NY,1,msm valid certification state ltc administrat...,None,None
218,218,218,Sr. Business Intelligence Analyst - Pop. Healt...,https://www.indeed.com/rc/clk?jk=37a8fbd067108...,Johns Hopkins Health System,"Baltimore, MD 21201 (Downtown area)+2 locations",30+ days ago,2022-05-23,Advanced knowledge of medical terminology and ...,3.4,...,0,None,None,0,1,MD,1,strengthen team success required certificatio...,None,data collection reporting data information di...
289,289,289,Population Health RE Data Analyst (Clinical Bu...,https://www.indeed.com/company/NYC-Health-+-Ho...,NYC Health + Hospitals,"New York, NY",17 days ago,2022-05-23,Extracting and analyzing health/clinical data ...,3.8,...,0,experience satisfactory combination education ...,None,0,1,NY,1,None,None,None
306,306,306,Clinical Informatics RN,https://www.indeed.com/rc/clk?jk=74827870c655e...,Barton HealthCare System,"South Lake Tahoe, CA 96150",30+ days ago,2022-05-23,Occasional travel to various health system loc...,3.7,...,1,clinical application assigned user end user r...,None,0,1,CA,1,inpatient preferred current rn clinical infor...,None,None
337,337,337,Collaborative Biostatistician I,https://www.indeed.com/rc/clk?jk=de5a8260322dc...,Kaiser Permanente,"Seattle, WA 98101 (Downtown area)+1 location",10 days ago,2022-05-23,Experience with medical or health research and...,4.1,...,1,completes federal local compliance support de...,None,0,1,WA,1,statistic time employment start certification...,None,None
354,354,354,Business Development Analyst,https://www.indeed.com/rc/clk?jk=d837f3132fa33...,Conway Regional Heath System,"Conway, AR 72034",3 days ago,2022-05-23,Gathers and evaluates data and provides suppor...,NaN,...,0,None,None,0,1,AR,0,None,None,None


In [30]:
# Jobs with College graduate degrees requested
df_updated[df_updated['graduate']==1]

,Unnamed: 0,Unnamed: 0.1,Job_titles,Job_URLS,Company,location,PostDate,ExtractDate,Summary,Job_ratings,...,otherDegree,trainings,degrees,remote,college,states,certificationFlag,license,programming,visualization
2,2,2,Clinical Informatics,https://www.indeed.com/rc/clk?jk=e1a9ee41cd34a...,Innovacer,Remote,12 days ago,2022-05-23,We are looking for a Clinical Informatics who ...,NaN,...,1,None,None,1,1,0,1,None,None,None
10,10,10,Health Informatics and Clinical Terminology Sp...,https://www.indeed.com/rc/clk?jk=56eb316ea1a9f...,ClosedLoop,"Remote in Austin, TX 78704",3 days ago,2022-05-23,"Bachelor's degree in related health field, e.g...",4.3,...,1,None,None,1,1,TX,0,None,None,None
14,14,14,Data Analyst - CCHS Health Informatics Clinica...,https://www.indeed.com/rc/clk?jk=0725cd4e5d932...,The University of Alabama,"Tuscaloosa, AL",10 days ago,2022-05-23,Required Department Minimum Qualifications: Ma...,4.3,...,1,report assist testing application end user em...,None,0,1,AL,0,None,None,None
15,15,15,Informatics Analyst,https://www.indeed.com/rc/clk?jk=43c8ff54b1e68...,Aspen Valley Hospital,"Aspen, CO 81611+3 locations",30+ days ago,2022-05-23,The Informatics Analyst develops and helps to ...,4.0,...,1,education supporting end user enhanced utiliz...,None,0,1,CO,1,None,None,None
16,16,16,Clinical Informatics Specialist,https://www.indeed.com/rc/clk?jk=49cf0b5880459...,Oregon Health & Science University,"Remote in Portland, OR 97239",15 days ago,2022-05-23,The department will apply the science of infor...,3.9,...,1,future state coordinate develops plan user do...,clinical healthcare environment associate rel...,1,1,OR,1,implement solution registration certification ...,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814,814,814,Public Health Informaticist,https://www.indeed.com/rc/clk?jk=be30904d64b13...,Weber County,"Ogden, UT 84401",30+ days ago,2022-05-23,Ability to integrate and incorporate informati...,2.7,...,0,None,None,0,1,UT,0,None,None,None
815,815,815,"Manager, Accounts Receivable",https://www.indeed.com/rc/clk?jk=ae3c98ebc591c...,Abbott Laboratories,"Orlando, FL",13 days ago,2022-05-23,Health care and well-being programs including ...,4.0,...,0,grow great career offer career development on...,None,0,1,FL,0,None,None,None
820,820,820,Epidemiology Program Manager (Masters),https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,PMHCC Inc.,"Philadelphia, PA+2 locations",30+ days ago,2022-05-23,"Epidemiology, statistics, or informatics: 7 ye...",3.8,...,0,covid response philadelphia coordinate dcc st...,None,0,1,PA,0,None,None,stream dcc produce report supply covid case d...
823,823,823,Senior Fullstack Software Developer,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Boston Children's Hospital,"Boston, MA+1 location",30+ days ago,2022-05-23,Building custom API servers to provide data to...,4.1,...,0,None,None,0,1,MA,0,None,None,None


In [57]:
# Jobs that requests bachelors degrees
df_updated[df_updated['bachelor']==1]

,Unnamed: 0,Unnamed: 0.1,Job_titles,Job_URLS,Company,location,PostDate,ExtractDate,Summary,Job_ratings,...,otherDegree,trainings,degrees,remote,college,states,certificationFlag,license,programming,visualization
0,0,0,Clinical Informatics Specialist - Contractor Role,https://www.indeed.com/company/Galen-Healthcar...,Galen HealthCare Solutions,Remote,24 days ago,2022-05-23,Serves as a resource person for daily operatio...,2.8,...,0,program support end user assist application i...,None,1,1,0,0,None,None,None
1,1,1,Electronic Health Record (EHR) Adoption/Optimi...,https://www.indeed.com/company/DEEPMILE/jobs/E...,DEEPMILE,Remote,3 days ago,2022-05-23,Knowledge of various specialized areas of heal...,NaN,...,0,electronic health record ehr learning support...,None,1,1,0,0,None,None,None
3,3,3,Informatics & Training Analyst,https://www.indeed.com/rc/clk?jk=c37cd6c881a7a...,Centura Health,"Centennial, CO 80112+1 location",30+ days ago,2022-05-23,"Informatics and Training Analyst supports, com...",3.8,...,0,minimal supervision epic credentialed hired s...,None,0,1,CO,0,None,None,None
4,4,4,Clinical Informatics Consultant (Generalist),https://www.indeed.com/company/Edera-L3C/jobs/...,Edera L3C,Remote,5 days ago,2022-05-23,The Clinical Informatics Consultant works dire...,NaN,...,0,implementation workflow development testing c...,capability education experience year healthca...,1,1,0,1,None,None,None
5,5,5,Clinical Informatics Epic Support Specialist (...,https://www.indeed.com/rc/clk?jk=8013a22efa6d1...,Providence,"Remote in Beaverton, OR 97075+21 locations",2 days ago,2022-05-23,Ensures that workflow and training knowledge d...,3.9,...,0,electronic health record ehr competency valid...,None,1,1,OR,0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817,817,817,Data Engineer (Informatics Specialist),https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Acacia Network,"Bronx, NY",30+ days ago,2022-05-23,Experience with electronic health record syste...,NaN,...,0,quantified maintained work ehr develop high q...,None,0,1,NY,0,None,None,None
819,819,819,Registered Dietitian 1,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Sodexo,"Yuma, AZ 85364",30+ days ago,2022-05-23,Have a working knowledge of EHR charting (EPIC...,3.7,...,0,None,None,0,1,AZ,1,active registered dietitian status required s...,None,None
823,823,823,Senior Fullstack Software Developer,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Boston Children's Hospital,"Boston, MA+1 location",30+ days ago,2022-05-23,Building custom API servers to provide data to...,4.1,...,0,None,None,0,1,MA,0,None,None,None
824,824,824,"Data Engineer, Insurance , Full time permanent",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"Villanti and Company,","San Diego, CA",6 days ago,2022-05-23,Well established and growing Insurance Company...,NaN,...,0,None,quicksight tableau powerbi education computer...,0,1,CA,0,None,None,warehouse design experience data tool quicksi...


In [58]:
# Flag the job if a certification is requested
def checkCertificationLenght(certification):
    if certification is None:
        return 0
    return len(certification)
df_updated['certificationFlag']=df_updated.apply(lambda x : checkCertificationLenght(x['certifications']),axis=1)

In [59]:
df_updated[df_updated['certificationFlag']>0]

# The reason why 40% of the jobs listed prefering or requiring certification while 
# the jobs with specifically mentioned certifications are almost 10% to 15% is that
# most of the jobs either mentioned that certification is required or prefered without specifying like:
# "Certification relevant to training," "Industry designation certifications," "Industry-focused Certifications,"
# or "Speciality certification is required", etc. Also, some jobs mentioned "Certification support" as part 
# of the professional development benefits in the job to attract candidates focusing on continous improvement

# Search for Certified Professional in Health Information & Management Systems (CPHIMS) through HIMSS, Certified Professional Healthcare Informatics (CPHI) through AHIMA, Certified Clinical Informatics through ANCC 
# ANCC 

,Unnamed: 0,Unnamed: 0.1,Job_titles,Job_URLS,Company,location,PostDate,ExtractDate,Summary,Job_ratings,...,otherDegree,trainings,degrees,remote,college,states,certificationFlag,license,programming,visualization
2,2,2,Clinical Informatics,https://www.indeed.com/rc/clk?jk=e1a9ee41cd34a...,Innovacer,Remote,12 days ago,2022-05-23,We are looking for a Clinical Informatics who ...,NaN,...,0,None,None,1,1,0,759,None,None,None
4,4,4,Clinical Informatics Consultant (Generalist),https://www.indeed.com/company/Edera-L3C/jobs/...,Edera L3C,Remote,5 days ago,2022-05-23,The Clinical Informatics Consultant works dire...,NaN,...,0,implementation workflow development testing c...,capability education experience year healthca...,1,1,0,1056,None,None,None
9,9,9,Clinical Informatics Consultant (Generalist),https://www.indeed.com/rc/clk?jk=1b348001de7b3...,National Coordination Center,"Remote in Washington, DC",4 days ago,2022-05-23,The Clinical Informatics Consultant works dire...,NaN,...,0,implementation workflow development testing c...,capability education experience year healthca...,1,1,WA,1056,None,None,None
15,15,15,Informatics Analyst,https://www.indeed.com/rc/clk?jk=43c8ff54b1e68...,Aspen Valley Hospital,"Aspen, CO 81611+3 locations",30+ days ago,2022-05-23,The Informatics Analyst develops and helps to ...,4.0,...,0,education supporting end user enhanced utiliz...,None,0,1,CO,892,None,None,None
21,21,21,Health Informatics Coordinator - Full Time Days,https://www.indeed.com/company/Prime-Healthcar...,Prime Healthcare,"Woonsocket, RI 02895 (East Woonsocket area)",10 days ago,2022-05-23,Knowledge of clinical informatics required.,3.4,...,0,platform software required experience ehr doc...,None,0,0,RI,801,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
821,821,821,Medical Pulmonary Nurse,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Sunrise Hospital,"Las Vegas, NV 89109+1 location",9 days ago,2022-05-23,Basic computer and hospital informatics skills...,3.9,...,0,None,None,0,0,NV,774,practice nursing independently maintains acqu...,None,None
822,822,822,NICU Registered Nurse,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Sunrise Hospital,"Las Vegas, NV 89109+1 location",12 days ago,2022-05-23,Basic computer and hospital informatics skills...,3.9,...,0,None,None,0,0,NV,889,practice nursing independently maintains acqu...,None,None
825,825,825,Physician Informaticist - Inpatient Focus,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Centura Health,"Centennial, CO 80112+1 location",30+ days ago,2022-05-23,Explores the use of medical informatics system...,3.8,...,0,management process oversees documentation bri...,None,0,0,CO,897,ehr preferred current unrestricted practice m...,None,None
827,827,827,Senior Data Engineer - TS/SCI w/Poly,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,General Dynamics,"Remote in McLean, VA",30+ days ago,2022-05-23,"Bachelor’s degree in information systems, info...",3.8,...,0,None,None,1,1,VA,638,None,None,None


In [60]:
# Flag the job if a certification is requested
def checkLicenseLenght(license):
    if license is None:
        return 0
    return len(license)
df_updated['licenseFlag']=df_updated.apply(lambda x : checkCertificationLenght(x['license']),axis=1)

In [61]:
df_updated[df_updated['licenseFlag']>0]

,Unnamed: 0,Unnamed: 0.1,Job_titles,Job_URLS,Company,location,PostDate,ExtractDate,Summary,Job_ratings,...,trainings,degrees,remote,college,states,certificationFlag,license,programming,visualization,licenseFlag
16,16,16,Clinical Informatics Specialist,https://www.indeed.com/rc/clk?jk=49cf0b5880459...,Oregon Health & Science University,"Remote in Portland, OR 97239",15 days ago,2022-05-23,The department will apply the science of infor...,3.9,...,analysis design development testing end user ...,clinical healthcare environment associate rel...,1,1,OR,0,implement solution registration certification ...,None,None,1022
23,23,23,Ambulatory Clinical Informatics - RN (COOL Team),https://www.indeed.com/rc/clk?jk=ab548484d3ce3...,CEDARS-SINAI,"Los Angeles, CA",30+ days ago,2022-05-23,"Solid understanding with clinical, financial a...",4.1,...,None,None,0,1,CA,0,experience training required current register...,None,None,862
25,25,25,Clinical Terminologist,https://www.indeed.com/rc/clk?jk=292d38fc4d465...,Cerner Corporation,Remote in Missouri,15 days ago,2022-05-23,Some Cerner positions may be obligated to comp...,3.5,...,None,None,1,1,MO,939,entire country cerner solution facility count...,None,None,785
28,28,28,Public Health Informatics Specialist,https://www.indeed.com/rc/clk?jk=5627790507cc2...,State Department of Health,"Oklahoma City, OK",11 days ago,2022-05-23,"Bachelor’s degree in Health Informatics, Healt...",3.9,...,None,None,0,1,OK,0,note requirement education certification requ...,None,None,997
31,31,31,Data Scientist - Healthcare Quality Informatic...,https://www.indeed.com/company/Blue-Sky-Techno...,BlueSky Technology Solutions,Remote,16 days ago,2022-05-23,This permanent position is 100% remote and off...,NaN,...,None,None,1,0,0,819,preferred sql year preferred certification ep...,None,None,828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
821,821,821,Medical Pulmonary Nurse,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Sunrise Hospital,"Las Vegas, NV 89109+1 location",9 days ago,2022-05-23,Basic computer and hospital informatics skills...,3.9,...,None,None,0,0,NV,774,practice nursing independently maintains acqu...,None,None,933
822,822,822,NICU Registered Nurse,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Sunrise Hospital,"Las Vegas, NV 89109+1 location",12 days ago,2022-05-23,Basic computer and hospital informatics skills...,3.9,...,None,None,0,0,NV,889,practice nursing independently maintains acqu...,None,None,933
825,825,825,Physician Informaticist - Inpatient Focus,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Centura Health,"Centennial, CO 80112+1 location",30+ days ago,2022-05-23,Explores the use of medical informatics system...,3.8,...,management process oversees documentation bri...,None,0,0,CO,897,ehr preferred current unrestricted practice m...,None,None,925
826,826,826,"RN, Registered Nurse (Primary Care - Broomall)",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Children's Hospital of Philadelphia,"Broomall, PA 19008+3 locations",12 days ago,2022-05-23,Demonstrates health care informatics competenc...,4.5,...,required program applies education internal s...,None,0,0,PA,0,previous supervisory experience preferred cer...,None,None,1124


In [27]:
# Checking the location of the jobs by state

# Checking jobs that have the flexibility of working remotely
def checkRemote(location):
    if 'remote' in location.lower():
        return 1
    else:
        return 0
df_updated['remote']=df_updated.apply(lambda x : checkRemote(x['location']),axis=1)

In [28]:
statesAbbrev=list(us_state_abbrev.values())
statesAb=[]
for st in statesAbbrev:
    statesAb.append(" "+st)
states=list(us_state_abbrev.keys())

In [29]:
def checkState(location):
    for st in statesAb:
        if st in location:
            return st.strip()
    for st in states:
        if st in location:
            return us_state_abbrev[st]
    return '0'
df_updated['states']=df_updated.apply(lambda x : checkState(x['location']),axis=1)

In [30]:
df_updated['states'].value_counts().to_csv('States.csv')

### N.B. 1/4 of the jobs have a remote working flexibility

In [ ]:
df_updated[df_updated['remote']==1]

### Checking job flexibility with college degree requirement (no correlation found)

In [81]:
df_updated['remote'].corr(df_updated['college'])

-0.07909216195943626

### Checking the skills, certifications, data visualization tools in highest demand

In [142]:
def getHighDemandJobDescription(df,demand):
    def preprocess(text):
        tokens = []
        tokenizer = nltk.tokenize.RegexpTokenizer(r"[a-z]+")
        for token in tokenizer.tokenize(str(text)):
            tokens.append(token)
        return tokens
    if demand=='skills':
        List=df['skills'].tolist()
    elif demand=='certifications':
        List=df['certifications'].tolist()
    elif demand=='trainings':
        List=df['trainings'].tolist()
    elif demand=='license':
        List=df['license'].tolist()
    elif demand=='programming':
        List=df['programming'].tolist()
    elif demand=='visualization':
        List=df['visualization'].tolist()
    else:
        List=df['degrees'].tolist()
    processed = list(map(preprocess, List))
    fdist = nltk.FreqDist([token for doc in processed for token in doc])
    print(fdist.tabulate(1000))

In [ ]:
# Get high demand skills
getHighDemandJobDescription(df_updated,'skills')
# Get high demand certifications
getHighDemandJobDescription(df_updated,'certifications')
# Top university degrees listed 
getHighDemandJobDescription(df_updated,'degrees')

In [ ]:
# Get high demand data visualization and programming tools
import numpy as np
df=df_updated
df['visualization'].replace('', np.nan).dropna(inplace=True)
print(df)
getHighDemandJobDescription(df,'visualization')

In [ ]:
maskSkill1 = df_updated['cleanDesc'].str.contains(r'SQL', na=True)
df_updated[maskSkill1]

# The number of jobs requesting each skill:
# Analytics and Problem Sovling 623
# Communication skills 337
# Oral Communication 238
# Project Management 199
# Statistics 198
# Interpersonal skills 172
# Critical thinking 67

# The top mentioned tools
# Excel 514
# SQL 150

# Tool listed tools

#mask6 = df_updated['cleanDesc'].str.contains(r'js', na=True) & df_updated['cleanDesc'].str.contains(r'certificat', na=True)
#mask6 = df_updated['cleanDesc'].str.contains(r'critical thinking', na=True) & df_updated['graduate']==1
#mask6 = df_updated['cleanDesc'].str.contains(r'java', na=True)
#mask6 = df_updated['Job_description'].str.contains(r' R ', na=True)
#mask6 = df_updated['cleanDesc'].str.contains(r'power bi', na=True)
df_updated[mask6]
# Python 74
# Tableau 73
# Java 29
# R 24
# SPSS 14
# Power BI 13


In [62]:
# List the top listed keywords for certifications in spreadsheet
CertificationsWording_list = df_updated["certifications"].tolist()
certificationDictionary={}
for certificationWording in CertificationsWording_list:
    if certificationWording is None:
        continue
    l= certificationWording.split()
    for w in l:
        if w in certificationDictionary:
            certificationDictionary[w]+=1
        else:
            certificationDictionary[w]=1

In [63]:
with open('certificatesKeywords.csv','w') as file:
    line="word,count"
    file.write(line)
    file.write('\n')
    for k in certificationDictionary:
        line=k+","+str(certificationDictionary[k])
        file.write(line)
        file.write('\n')

In [82]:
# checking in how many jobs certain certifications was listed
mask3 = df_updated['cleanDesc'].str.contains(r'hims', na=True)
df_updated[mask3]

,Unnamed: 0,Unnamed: 0.1,Job_titles,Job_URLS,Company,location,PostDate,ExtractDate,Summary,Job_ratings,...,trainings,degrees,remote,college,states,certificationFlag,license,programming,visualization,licenseFlag
85,85,85,Health Informatics Analyst - RN (Full Time),https://www.indeed.com/rc/clk?jk=9601bf551ddaf...,Sauk Prairie Healthcare,"Prairie du Sac, WI 53578",30+ days ago,2022-05-23,Competitive health and dental insurance options.,4.3,...,initially potentially ongoing user education ...,health informatics health alternative minimum...,0,1,WI,1048,required valid state wisconsin registered nur...,None,None,856
264,264,264,"Open Rank Clinical Faculty, Health Information...",https://www.indeed.com/rc/clk?jk=df653db193325...,Texas State University,"Round Rock, TX 78665",17 days ago,2022-05-23,An earned masters or doctorate degree in healt...,4.4,...,None,required provide official transcript granting...,0,1,TX,856,None,None,None,0
413,413,413,CLINICAL INFORMATIC SPECIALIST-CLINICAL INFORM...,https://www.indeed.com/rc/clk?jk=289a34f858d5e...,Union Health,"Terre Haute, IN 47804",30+ days ago,2022-05-23,Prefer a Master’s of Science in Nursing Inform...,2.0,...,safety experience responsible delivering educ...,None,0,1,IN,949,None,None,None,0
620,620,620,PHARMACY INFORMATICS ANALYST- EAMC PHARMACY,https://www.indeed.com/rc/clk?jk=ce8ae41a39de4...,East Alabama Medical Center,"Opelika, AL 36801",5 days ago,2022-05-23,"The Pharmacy Informatics Analyst builds, imple...",4.0,...,operation issue change request related medica...,information computer science similar related ...,0,1,AL,922,analyst role preferred registration certifica...,None,None,1016
788,788,788,Clinical Informatics Analyst,https://www.indeed.com/rc/clk?jk=3a02f7e4353ba...,Central Peninsula Hospital,"Soldotna, AK 99669",26 days ago,2022-05-23,Advanced Degree in Healthcare Informatics rela...,3.7,...,software support maintenance troubleshooting ...,industry terminology preferred advanced healt...,0,1,AK,862,None,None,None,0


In [95]:
# checking in how many jobs certain certifications that were listed
mask3 = df_updated['certifications'].str.contains(r'coder',na=False)
df_updated[mask3]

,Unnamed: 0,Unnamed: 0.1,Job_titles,Job_URLS,Company,location,PostDate,ExtractDate,Summary,Job_ratings,...,trainings,degrees,remote,college,states,certificationFlag,license,programming,visualization,licenseFlag
373,373,373,Remote Lead,https://www.indeed.com/rc/clk?jk=8cda48609d3cd...,Northwestern Medicine,"Remote in Chicago, IL",30+ days ago,2022-05-23,"At Northwestern Medicine, we pride ourselves o...",3.9,...,issue reported end user document tip sheet st...,None,1,1,IL,868,None,None,None,0
394,394,394,Remote Coding Specialist II,https://www.indeed.com/rc/clk?jk=3126d25924eb5...,Northwestern Medicine,"Remote in Chicago, IL 60611+3 locations",30+ days ago,2022-05-23,"At Northwestern Medicine, we pride ourselves o...",3.9,...,maintains coding reference information physic...,None,1,1,IL,868,None,None,None,0
568,568,568,Remote Operations Coordinator,https://www.indeed.com/rc/clk?jk=531e4b08ae2c1...,Northwestern Medicine,"Remote in Chicago, IL 60611+3 locations",30+ days ago,2022-05-23,"At Northwestern Medicine, we pride ourselves o...",3.9,...,maintains coding reference information physic...,None,1,1,IL,868,None,None,None,0
707,707,707,"Billing & Coding Analyst, Full Time, MSS Miram...",https://www.indeed.com/rc/clk?jk=0a8d3fba60782...,Memorial Healthcare System,"Miramar, FL 33025+1 location",30+ days ago,2022-05-23,"Reviews, analyzes, and coordinates charge mast...",4.1,...,None,None,0,1,FL,810,None,None,None,0


In [65]:
# List the top listed keywords for certifications in spreadsheet
licenseWording_list = df_updated["license"].tolist()
licenseDictionary={}
for licenseWording in licenseWording_list:
    if licenseWording is None:
        continue
    l= licenseWording.split()
    for w in l:
        if w in licenseDictionary:
            licenseDictionary[w]+=1
        else:
            licenseDictionary[w]=1

In [66]:
with open('licensesKeywords.csv','w') as file:
    line="word,count"
    file.write(line)
    file.write('\n')
    for k in licenseDictionary:
        line=k+","+str(licenseDictionary[k])
        file.write(line)
        file.write('\n')

In [67]:
# checking in how many jobs certain licenses that were listed
mask3 = df_updated['license'].str.contains(r'epic',na=False)
df_updated[mask3]

,Unnamed: 0,Unnamed: 0.1,Job_titles,Job_URLS,Company,location,PostDate,ExtractDate,Summary,Job_ratings,...,trainings,degrees,remote,college,states,certificationFlag,license,programming,visualization,licenseFlag


In [109]:
# The average indeed ratings of Institutions that listed the jobs
print(df_updated.agg({"Job_ratings": ["mean"]}))
print(len(df_updated.Company.unique()))

      Job_ratings
mean     3.730213
592


In [103]:
# Save the results of analysis in csv
df_updated.to_csv('JobDetailsAnalysis.csv')